In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
import sys
sys.path.insert(0, "../")
from koala.preprocessing.lemmatizer import lemmatize_text
from koala.preprocessing.vocabulary_reductor import VocabularyReductor

/home/rodri/PycharmProjects/ClasificacionSinMuestras/venv/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [4]:
from spacy.lang.es.stop_words import STOP_WORDS


# Data loading

In [5]:
pd_data = pd.read_csv("../data/HISTORICAL_ PE_PlayConsole_stats.xlsx - reviews.csv")[["Star Rating", "Review Text"]].dropna()

/home/rodri/PycharmProjects/ClasificacionSinMuestras/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
len(pd_data)

27254

In [7]:
texts = pd_data["Review Text"].values
targets = pd_data["Star Rating"].values

In [8]:
print(len(texts))
texts_indices = [i for i, t in enumerate(texts) if ((len(t.split()) > 6))]
texts = [texts[i] for i in texts_indices]
targets = [targets[i] for i in texts_indices]
print(len(texts))

27254
17078


In [9]:
texts = [lemmatize_text(t) for t in tqdm(texts)] 


100%|██████████| 17078/17078 [54:32<00:00,  5.22it/s] 


In [10]:
vr = VocabularyReductor("surveys_peru").get_vocab_reductor(texts)

Building word embedding
Vocab reduction function crated with 124 terms in dict
Building word embedding
Vocab reduction function crated with 27 terms in dict


In [11]:
import random
for _ in range(5):
    i = random.randint(0,len(texts))
    print(texts[i])
    print(vr(texts[i]))
    print("===========================")

es un app super seguro ya que contar con el tecnología de huella digital , también es muy útil ya que tener un tecnología mas agil y eficaz ya que no es necesario salir y volver para escoger el servicio para pagar o transferir , puedes realizar todo des un solo pantalla . estoy muy contenta , sugerir que colocar el opción de reclamo y estaría perfecto .
es un app muy seguro ya que contar con el tecnología de huella digital , también es muy útil ya que tener un tecnología más agil y eficaz ya que no es necesario salir y volver para escoger el servicio para pagar o transferir , puedes hacer todo des un solo pantalla , estoy muy contenta , sugerir que colocar el opción de reclamo y estaría perfecto ,
desde que se actualizar no me permitir abrir el aplicativo por qué ahí mismo se cerrar , tener que estar desinstalar e instalar parq poder ingresar ... pero solo me permitir un vez tener acceso , luego persisto con el mismo problema de no poder ingresar .
desde que se actualizar no me dejar i

In [12]:
texts = [vr(t) for t in texts]

In [13]:
import pickle

with open('lemmatized_texts.pickle', 'wb') as handle:
    pickle.dump(texts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
import pickle
with open('lemmatized_texts.pickle', 'rb') as handle:
    texts = pickle.load(handle)

In [200]:
from spellchecker import SpellChecker
spell = SpellChecker(language='es')  # use the Spanish Dictionary
DICT_MISSPELLED = {}
def correct_misspelled(text):
    misspelled = spell.unknown(text.split())
    for t in misspelled:
        if t in DICT_MISSPELLED.keys():
            rep = DICT_MISSPELLED[t]
        else:
            rep = spell.correction(t)
            DICT_MISSPELLED[t] = rep
        text = text.replace(t, rep)
    return text
def is_correct(text):
    return len(list(spell.unknown(text.split()))) == 0


In [44]:
print(texts[:5])

['Solo me sirve para ver mis movimientos, pero desde que cambié de equipo no me permite activar el Token Digital, a pesar que ya actualicé mi información en el cajero automático. Si no te permite hacer transferencias, es una App innecesaria.', 'Fácil manejo de operaciones y control detallado de ingresos y gastos. Ahorro de tiempo y control eficiente de tus finanzas personales.', 'Sres. en la aplicación me sale en estos momentos no podemos atenderlo cuando porno mi numero DNI y contraseña 😠😠😠', 'Todo ok hasta que cambie de celular. Cuando ingreso a activar el token digital, nunca me envía el sms . Deben corregir eso.', 'Monse xq no puedo activar mi toque digital']


In [45]:
from functools import reduce
special_chars = {".": " ", ",": " ", 
                 "(": " ", 
                 ")": " ",
                 "!": " ",
                 "¡": " ",
                 "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "U"}
def preprocess(text):
    #text = correct_misspelled(text)
    text = reduce(lambda t, c: t.replace(c[0], c[1]), special_chars.items(), text)
    return text
texts = [preprocess(t) for t in tqdm(texts)]

100%|██████████| 17078/17078 [00:00<00:00, 339800.68it/s]


In [46]:
from gensim.models.phrases import Phrases
phrase_model = Phrases([t.split() for t in texts], min_count=10, threshold=2.5)

In [47]:

def tokenizer(text):
    x = text.split()
    for _ in range(2):
        x = phrase_model[x]
    return x
for i in range(20,25):
    print(tokenizer(texts[i]))

['Es_muy', 'util', 'y', 'rapido', 'el_servicio', 'banca_movil']
['Hace', '5', 'meses', 'que', 'no_puedo', 'ingresar', 'y', 'no', 'me_dan', 'solucion', 'respectuva', 'fui', 'agencia', 'y', 'me_dicen', 'que', 'hay', 'error', 'de', 'sistema', 'que', 'llame_a', 'la', 'operadora', 'vulvo', 'a', 'llamar', 'y', 'siempre', 'me_dicen', 'los', 'mimso', 'que_vuelva', 'a_intentar', 'en_unos', 'dias', 'que', 'es', 'por', 'eror', 'de', 'sistema', 'y', 'hasta', 'el_dia', 'de', 'hoy', 'persistente', 'el_problema']
['Porqueria_de', 'aplicacion', 'no', 'me_deja', 'realizar_ninguna', 'operacion', 'nisiquiera', 'una_transferencia', 'voy_a', 'sacar', 'todo', 'mi_dinero', 'y', 'a', 'cancelara', 'esta', 'porqueria', 'se_supone', 'q', 'la_app', 'es', 'para', 'evitarse', 'ir_al', 'banco', 'a', 'hacer', 'grandes', 'colas', 'pero', 'no', 'estos', 'grandisimos', 'animales', 'nose', 'sabe', 'que', 'hacen', 'pero', 'sienpre', 'la', 'cagan', 'ni', 'mas', 'con_este', 'banco', 'los', 'denunciare', 'a', 'Indecopi']
['T

In [48]:
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(texts, targets, train_size=0.85)


# Feature generation

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=20, ngram_range=(1, 2),
                    stop_words=STOP_WORDS.union({"excelente", "bueno"}))
X_train = cv.fit_transform(texts_train)
X_train.shape

(14516, 1379)

In [50]:
X_test = cv.transform(texts_test)

# Linear Regressor

In [51]:
lr = LinearRegression().fit(X_train, y_train)

In [52]:
print("Training score: ", r2_score(y_train, lr.predict(X_train)))
print("Test score: ", r2_score(y_test, lr.predict(X_test)))

Training score:  0.3473681075270456
Test score:  0.20486613376951912


In [53]:
n_important_words = 20
print("MOST IMPORTANT {} words".format(n_important_words))
sorted(list(list(zip(cv.get_feature_names(), lr.coef_))), key=lambda x: abs(x[1]), reverse=True)[:n_important_words]

MOST IMPORTANT 20 words


[('encanta', 1.955477392068662),
 ('exelente', 1.8304742446494335),
 ('sigan', 1.7458088286119544),
 ('doy estrellas', 1.2464119075606659),
 ('facilita', 1.1885046366618561),
 ('segura', 1.0582534544631914),
 ('play', 0.9633108643893294),
 ('genial', 0.9555125310392966),
 ('den solucion', 0.9495324442317168),
 ('activacion digital', -0.9328070390246064),
 ('media hora', -0.9231063445794985),
 ('recomendable', 0.919014712928766),
 ('util', 0.8547129634693594),
 ('perfecto', 0.8535691562620091),
 ('atencion', 0.8473435309042614),
 ('mejorado', 0.8457907143202277),
 ('seguro', 0.8407893835642324),
 ('ok', 0.8372424555585958),
 ('programa', 0.8252521922097962),
 ('gracias bbva', 0.8243432143810956)]

# OMP

In [54]:
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X_train_norm = X_train.toarray()
X_test_norm = X_test.toarray()
higher_norm = max([np.linalg.norm(X_train_norm[:, i]) for i in range(X_train.shape[1])])
       
for i in range(X_train.shape[1]):
    X_train_norm[:, i] = X_train_norm[:, i] * higher_norm / np.linalg.norm(X_train_norm[:, i])
    X_test_norm[:, i] = X_test_norm[:, i] * higher_norm / np.linalg.norm(X_test_norm[:, i])

/home/rodri/PycharmProjects/ClasificacionSinMuestras/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [63]:
from sklearn.model_selection import GridSearchCV

omp = OrthogonalMatchingPursuit(n_nonzero_coefs=50)

gcv = GridSearchCV(omp, param_grid={"n_nonzero_coefs": range(50, 300, 50)}, scoring="r2").fit(X_train_norm, y_train)

In [64]:
gcv.best_params_

{'n_nonzero_coefs': 200}

In [65]:
print("Training score: ", r2_score(y_train, gcv.predict(X_train.toarray())))
print("Test score: ", r2_score(y_test, gcv.predict(X_test.toarray())))

Training score:  0.09177386908572549
Test score:  0.08293260478323128


In [66]:
sorted(list(filter(lambda x: x[1] != 0, list(zip(cv.get_feature_names(), gcv.best_estimator_.coef_)))), key=lambda x: abs(x[1]), reverse=True) 

[('pesima', -0.2076846555278469),
 ('actualizacion', -0.18625034369942498),
 ('pesimo', -0.18425513860407536),
 ('facil', 0.1712039785354156),
 ('encanta', 0.16060217293543716),
 ('version', -0.1601268673516938),
 ('gracias', 0.14805264906196025),
 ('mala', -0.1408227358618903),
 ('exelente', 0.1233216645997422),
 ('util', 0.12180585850288748),
 ('facilita', 0.11676866504373676),
 ('atencion', 0.10489373308676171),
 ('segura', 0.10338991059846014),
 ('genial', 0.09690124116149251),
 ('ingresar', -0.09283481803067897),
 ('sirve', -0.091718925232076),
 ('entrar', -0.08827533173482081),
 ('horrible', -0.08788241793768868),
 ('porqueria', -0.08772788681750622),
 ('seguro', 0.08639643613867401),
 ('lenta', -0.08259315176480392),
 ('sigan', 0.08188451088651937),
 ('ningun problema', 0.08160654676854219),
 ('recomiendo', 0.07990934338034064),
 ('app', -0.07812525961438788),
 ('rapida', 0.07697524602985212),
 ('casa', 0.07695056001326261),
 ('desastre', -0.07455845634153473),
 ('queda', -0.072